### Best function on Jazz

In [2]:
import numpy as np
import networkx as nx
from scipy.sparse.linalg import eigsh

def score_nodes(adj_matrix):
    adj_matrix = np.array(adj_matrix)
    G = nx.from_numpy_array(adj_matrix)
    LCC = G.subgraph(max(nx.connected_components(G), key=len))

    eigenvalues, eigenvectors = np.linalg.eig(adj_matrix)
    max_eigenvalue_index = np.argmax(eigenvalues)
    eigenvector = eigenvectors[:, max_eigenvalue_index]
    normalized_eigenvector = eigenvector / np.sum(eigenvector)

    ii = {v: i for i, v in enumerate(LCC.nodes())}
    L = nx.normalized_laplacian_matrix(LCC)
    eigenvalues, eigenvectors = eigsh(L, which='SM', maxiter=1000 * L.shape[0])
    Fiedler = eigenvectors[:, 1]

    H = nx.Graph([(u, v) for u, v in LCC.edges() if Fiedler[ii[u]] * Fiedler[ii[v]] <= 0.0])
    for v in H.nodes():
        H.nodes[v]['weight'] = 1.0 / H.degree(v)
    cover = list(nx.algorithms.approximation.min_weighted_vertex_cover(H))
    max_degree = max([G.degree(v) for v in G.nodes() if v not in cover])
    min_weight = min(H.nodes[v]['weight'] for v in H.nodes())
    scored_nodes = {}
    for v in G.nodes():
        if v in cover:
            scored_nodes[v] = H.nodes[v]['weight']
        else:
            scored_nodes[v] = normalized_eigenvector[v] * min_weight
    return scored_nodes

### Best function on Network_Science

In [ ]:
import networkx as nx

def score_nodes(adj_matrix):
    G = nx.Graph(adj_matrix)
    
    score1 = {node_id: 0.0 for node_id in G.nodes()}
    for u, v in G.edges():
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        score1[u] += 1.0 / degree_u if degree_u != 0 else 0.0
        score1[v] += 1.0 / degree_v if degree_v != 0 else 0.0
    
    betweenness = nx.betweenness_centrality(G)
    pagerank = nx.pagerank(G)
    
    weight_degree = 0.2
    weight_betweenness = 0.3
    weight_pagerank = 0.5
    
    scored_nodes = {}
    for node_id in G.nodes():
        score = (weight_degree * score1.get(node_id, 0) +
                 weight_betweenness * betweenness[node_id] +
                 weight_pagerank * pagerank[node_id])
        scored_nodes[node_id] = score
    
    return scored_nodes

### Best function on Twitter

In [ ]:
import numpy as np
import networkx as nx
from scipy.sparse.linalg import eigsh

def score_nodes(adj_matrix):
    G = nx.from_numpy_array(adj_matrix)
    score1 = {node_id: 0.0 for node_id in G.nodes()}
    for u, v in G.edges():
        score_u = score_v = 0.0
        degree_u = G.degree(u)
        degree_v = G.degree(v)
        if degree_u != 0:
            score_u = 1.0 / degree_u
        if degree_v != 0:
            score_v = 1.0 / degree_v
        score1[u] += score_u
        score1[v] += score_v
    betweenness = nx.betweenness_centrality(G)
    pagerank = nx.pagerank(G)
    closeness = nx.closeness_centrality(G)
    scored_nodes = {node_id: 0.0 for node_id in G.nodes()}
    for node_id in G.nodes():
        scored_nodes[node_id] = (0.2 * score1.get(node_id, 0) +
                           0.3 * betweenness[node_id] +
                           0.5 * pagerank[node_id] +
                           0.4 * closeness[node_id])  # Additional metric
    
    # Algorithm 2 modifications
    G1 = G
    G2 = nx.from_numpy_array(np.where(adj_matrix > 0, 1, 0))

    centrality1 = nx.betweenness_centrality(G1)
    centrality2 = nx.betweenness_centrality(G2, normalized=True)

    for node, centrality in centrality1.items():
        scored_nodes[node] += centrality + centrality2[node]

    LCC1 = G1.subgraph(max(nx.connected_components(G1), key=len))
    LCC2 = G2.subgraph(max(nx.connected_components(G2), key=len))

    L1 = nx.laplacian_matrix(LCC1)
    L2 = nx.laplacian_matrix(LCC2)

    eigenvalues1, eigenvectors1 = eigsh(L1.astype(np.float32), k=2, which='SM')
    eigenvalues2, eigenvectors2 = eigsh(L2.astype(np.float32), k=2, which='SM')

    Fiedler1 = eigenvectors1[:, 1]
    Fiedler2 = eigenvectors2[:, 1]

    H1 = nx.Graph()
    H2 = nx.Graph()

    for u, v in LCC1.edges():
        if Fiedler1[list(LCC1.nodes()).index(u)] * Fiedler1[list(LCC1.nodes()).index(v)] <= 0.0:
            H1.add_edge(u, v)

    for u, v in LCC2.edges():
        if Fiedler2[list(LCC2.nodes()).index(u)] * Fiedler2[list(LCC2.nodes()).index(v)] <= 0.0:
            H2.add_edge(u, v)

    for v in H1.nodes():
        scored_nodes[v] += 1.0 / H1.degree(v)

    for v in H2:
        degree = nx.degree(H2, v)
        scored_nodes[v] += np.log1p(1.0 / degree) if degree > 0 else 0

    cover1 = nx.algorithms.approximation.min_weighted_vertex_cover(H1)
    cover2 = nx.algorithms.approximation.min_weighted_vertex_cover(H2)

    for v in cover1:
        scored_nodes[v] += 1

    for v in cover2:
        scored_nodes[v] += np.log1p(nx.degree(G2, v))

    max_score = max(scored_nodes.values())
    min_score = min(scored_nodes.values())

    for node in scored_nodes:
        if max_score != min_score:
            scored_nodes[node] = (scored_nodes[node] - min_score) / (max_score - min_score)
        else:
            scored_nodes[node] = 0

    return scored_nodes

### Best function on Synthetic Network (BA)

In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigsh

def score_nodes(edge_matrix):
    G = nx.from_numpy_array(edge_matrix)
    max_pagerank = max(nx.pagerank(G).values())
    LCC = G.subgraph(max(nx.connected_components(G), key=len))
    ii = {v: i for i, v in enumerate(LCC.nodes())}
    L = nx.normalized_laplacian_matrix(LCC)
    eigenvalues, eigenvectors = eigsh(L.astype(np.float32), k=2, which='SM', maxiter=1000 * L.shape[0])
    Fiedler = eigenvectors[:, 1]
    H = nx.Graph([(u, v) for u, v in LCC.edges() if Fiedler[ii[u]] * Fiedler[ii[v]] <= 0.0])
    for v in H.nodes():
        H.nodes[v]['weight'] = 1.0 / H.degree(v)
    cover = list(nx.algorithms.approximation.min_weighted_vertex_cover(H, weight='weight'))
    max_degree = max([G.degree(v) for v in G.nodes() if v not in cover])
    min_weight = min(H.nodes[v]['weight'] for v in H.nodes())
    scored_nodes = {}

    for node_id in G.nodes:
        node_neighbors = G[node_id]
        norm_betweenness = sum([edge_matrix[node_id][neighbor] for neighbor in node_neighbors]) / np.sum(edge_matrix)
        score = (0.5 * norm_betweenness) + (0.5 * (nx.pagerank(G)[node_id] / max_pagerank))
        if node_id in cover:
            score += min_weight
        else:
            score *= G.degree[node_id] / max_degree
            score *= min_weight
        scored_nodes[node_id] = score

    pagerank_scores = nx.pagerank(G, weight='weight')
    betweenness = nx.betweenness_centrality(G)
    
    for node in G.nodes:
        node_neighbors = G[node]
        norm_betweenness = np.sum([(edge_matrix[node][neighbor] / np.sum(edge_matrix[neighbor])) for neighbor in node_neighbors])
        score = pagerank_scores[node] + (2 * np.log10(betweenness[node] + 1)) + (0.5 * norm_betweenness)
        scored_nodes[node] += score

    return scored_nodes